In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("iceberg_to_clickhouse")
    .master("spark://spark-master:7077")
    .config(
        "spark.jars.packages",
        ",".join([
            # s3 (AWS SDK v2)
            "software.amazon.awssdk:s3:2.25.28",
            "software.amazon.awssdk:sts:2.25.28",
            "software.amazon.awssdk:auth:2.25.28",
            "software.amazon.awssdk:aws-core:2.25.28",
            "software.amazon.awssdk:kms:2.25.28",
            # iceberg packages
            "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.10.0",
            # clcikhouse packages
            "com.clickhouse.spark:clickhouse-spark-runtime-3.4_2.12:0.8.0",
            "com.clickhouse:clickhouse-client:0.7.0",
            "com.clickhouse:clickhouse-http-client:0.7.0",
            "org.apache.httpcomponents.client5:httpclient5:5.2.1",
        ])
    )
    # iceberg configs
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.iceberg.catalog-impl", "org.apache.iceberg.rest.RESTCatalog")
    .config("spark.sql.catalog.iceberg.uri", "http://rest:8181")
    .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.iceberg.s3.path-style-access", "true")
    .config("spark.sql.catalog.iceberg.s3.access-key-id", "minio")
    .config("spark.sql.catalog.iceberg.s3.secret-access-key", "minio-password")
    # clickhouse configs
    .config("spark.sql.catalog.clickhouse", "com.clickhouse.spark.ClickHouseCatalog")
    .config("spark.sql.catalog.clickhouse.host", "clickhouse-server")
    .config("spark.sql.catalog.clickhouse.protocol", "http")
    .config("spark.sql.catalog.clickhouse.http_port", "8123")
    .config("spark.sql.catalog.clickhouse.user", "default")
    .config("spark.sql.catalog.clickhouse.password", "1234qwe")
    .config("spark.sql.catalog.clickhouse.database", "default")
    .config("spark.clickhouse.write.format", "json")
    
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
software.amazon.awssdk#s3 added as a dependency
software.amazon.awssdk#sts added as a dependency
software.amazon.awssdk#auth added as a dependency
software.amazon.awssdk#aws-core added as a dependency
software.amazon.awssdk#kms added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
com.clickhouse.spark#clickhouse-spark-runtime-3.4_2.12 added as a dependency
com.clickhouse#clickhouse-client added as a dependency
com.clickhouse#clickhouse-http-client added as a dependency
org.apache.httpcomponents.client5#httpclient5 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d93b32dd-8b7b-4441-a313-991836e3ef1c;1.0
	confs: [default]
	found software.amazon.awssdk#s3;2.25.28 in central
	found software.amazon.awssdk#aws-xml-protocol;2.25.28 in central
	found software.amazon.awssdk#aws-query-protocol;2.25.28 in cen

## Запись поверх iceberg

In [2]:
from pyspark.sql import Row
import random

offset = 10
count = 1000
data = [
    Row(id=i, name=f"test_record_{offset + i}", value=round(random.uniform(0, 1000), 2))
    for i in range(1, count + 1)
]

df = spark.createDataFrame(data)
df.show()

+---+--------------+------+
| id|          name| value|
+---+--------------+------+
|  1|test_record_11| 907.5|
|  2|test_record_12| 57.56|
|  3|test_record_13|968.93|
|  4|test_record_14|774.73|
|  5|test_record_15|818.73|
|  6|test_record_16|499.96|
|  7|test_record_17|936.79|
|  8|test_record_18| 484.3|
|  9|test_record_19|926.36|
| 10|test_record_20|963.62|
| 11|test_record_21|120.94|
| 12|test_record_22| 66.36|
| 13|test_record_23|467.74|
| 14|test_record_24|191.62|
| 15|test_record_25|636.19|
| 16|test_record_26|384.38|
| 17|test_record_27|190.62|
| 18|test_record_28|753.24|
| 19|test_record_29|472.41|
| 20|test_record_30|261.72|
+---+--------------+------+
only showing top 20 rows



In [3]:
df.repartition(10).writeTo("iceberg.test_schema.test_table").createOrReplace()

In [5]:
df_result = spark.table("iceberg.test_schema.test_table")
df_result.show()

+---+---------------+------+
| id|           name| value|
+---+---------------+------+
| 40| test_record_50| 38.93|
|463|test_record_473|278.15|
|337|test_record_347|126.52|
| 46| test_record_56|395.34|
| 89| test_record_99|979.13|
|172|test_record_182|848.57|
|340|test_record_350| 67.37|
|477|test_record_487|690.08|
|398|test_record_408|837.74|
| 25| test_record_35|310.83|
|353|test_record_363|771.76|
|295|test_record_305|146.51|
|474|test_record_484|210.33|
|450|test_record_460|490.26|
|  2| test_record_12| 57.56|
|323|test_record_333|742.96|
|241|test_record_251|689.68|
|107|test_record_117|424.88|
|298|test_record_308|458.58|
|297|test_record_307|534.06|
+---+---------------+------+
only showing top 20 rows



In [6]:
df_result.count()

1000

## Чтение из iceberg и запись в clickhouse

In [5]:
df = spark.table("iceberg.test_schema.test_table")
df.show(truncate=False)
df.printSchema()

+---+---------------+------+
|id |name           |value |
+---+---------------+------+
|40 |test_record_50 |38.93 |
|463|test_record_473|278.15|
|337|test_record_347|126.52|
|46 |test_record_56 |395.34|
|89 |test_record_99 |979.13|
|172|test_record_182|848.57|
|340|test_record_350|67.37 |
|477|test_record_487|690.08|
|398|test_record_408|837.74|
|25 |test_record_35 |310.83|
|353|test_record_363|771.76|
|295|test_record_305|146.51|
|474|test_record_484|210.33|
|450|test_record_460|490.26|
|2  |test_record_12 |57.56 |
|323|test_record_333|742.96|
|241|test_record_251|689.68|
|107|test_record_117|424.88|
|298|test_record_308|458.58|
|297|test_record_307|534.06|
+---+---------------+------+
only showing top 20 rows

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- value: double (nullable = true)



In [8]:
spark.sql("""
CREATE TABLE IF NOT EXISTS clickhouse.default.test_table (
    id INT NOT NULL,
    name STRING,
    value FLOAT
)
USING clickhouse
TBLPROPERTIES (
    'engine'='MergeTree()',
    'order_by'='id'
)
""")

DataFrame[]

In [6]:
df.repartition(10).writeTo("clickhouse.default.test_table").append()

In [7]:
df_new = spark.sql("select * from clickhouse.default.test_table")
df_new.show()

+---+--------------+------+
| id|          name| value|
+---+--------------+------+
|  1| test_record_1|787.86|
|  1|test_record_11| 907.5|
|  1|test_record_11| 907.5|
|  2| test_record_2|462.91|
|  2|test_record_12| 57.56|
|  2|test_record_12| 57.56|
|  3| test_record_3|534.92|
|  3|test_record_13|968.93|
|  3|test_record_13|968.93|
|  4| test_record_4|932.76|
|  4|test_record_14|774.73|
|  4|test_record_14|774.73|
|  5| test_record_5|282.25|
|  5|test_record_15|818.73|
|  5|test_record_15|818.73|
|  6| test_record_6| 11.39|
|  6|test_record_16|499.96|
|  6|test_record_16|499.96|
|  7| test_record_7|696.54|
|  7|test_record_17|936.79|
+---+--------------+------+
only showing top 20 rows



In [8]:
df_new.count()

3000

In [2]:
spark.stop()